# This notebook scans all directories in an input bucket and converts all files to wav

This occurs inplace and deletes the original file and replaces it with a .wav file, It overwrites an existing file by the same name.

It expects input of a "bucket" folder, consisting of subfolders named as species, and the audio files within.
If converting a single folder, create a parent folder to put it into.

Audio file formats are
- .mp3
- .flac
- .wav

Video file inputs are
- .mp4
- .m4a

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
import librosa, librosa.display
import pathlib
import glob
import pathlib
import subprocess

In [ ]:
import threading 
import time


In [ ]:
# A sample function to print squares
def print_squares(thread_name, numbers):

   for number in numbers:
       print(thread_name, number**2)
       
       # Produce some delay to see the output
       # syntax: time.sleep(<time in seconds : float>)
       time.sleep(1)

In [ ]:
thread_object = threading.Thread(target=<function name>, args=<tuple of arguments you want to pass>)

In [ ]:
# Creating 3 threads that execute the same function with different parameters
thread1 = threading.Thread(
   target=print_squares, args=("thread1", [1, 2, 3, 4, 5]))
   
thread2 = threading.Thread(
   target=print_squares, args=("thread2", [6, 7, 8, 9, 10]))
   
thread3 = threading.Thread(
   target=print_squares, args=("thread3", [11, 12, 13, 14, 15]))

In [ ]:
# Start the threads
thread1.start()
thread2.start()
thread3.start()

# Join the threads before moving further
thread1.join()
thread2.join()
thread3.join()

print("done all")

In [ ]:
# input parameters

source_folder ="/Volumes/UGREEN Samsung SSD/project echo audio/test data/test_1 wavs"

# options are currently ".mp3" or ".wav" (there is scope to add more)
output_file_type = ".wav"
input_file_type = ".mp3"




# TODO:
## Make sure to use try / except



In [ ]:
# check if there are multiple folders in source_folder
# source_folder ="/Volumes/UGREEN Samsung SSD/project echo audio/test data/test_1"

--- 
## Generate list of all folders in the source path

In [ ]:
sub_folders = pathlib.Path(source_folder).glob("**/*") #generator object
sub_folders
sub_folder_paths = [x for x in sub_folders if x.is_dir()]
sub_folder_paths


--- 
## Generate list of all audio files paths in the sub-folder

In [ ]:
# ### Testing
# sub_folder_paths[0] #carhorn
# audio_files = pathlib.Path(sub_folder_paths[0]).glob("**/*") #generator object
# audio_files

# audio_file_paths =  [x for x in audio_files if x.is_file()] #PosixPath list
# audio_file_paths


In [ ]:
def convert_file(input_file_path):
    parent_path = pathlib.PurePath(input_file_path).parent    
    file_name = pathlib.Path(input_file_path).stem   
    output_file_path = str(parent_path) + "/" + file_name + output_file_type
    # write the new wav file forcing overwrite 
    subprocess.call(['ffmpeg', '-y', '-i', input_file_path,
                 output_file_path],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.STDOUT)
    return

In [ ]:
# this function checks the filepath passed to it and checks the filetype
# if it is one of the listed filetypes ot be converted, it will make a new version 
# as a .wav and delete the original

def check_file_type(audio_file_path):  

    # check this file name and file type don't already exist
    if (pathlib.Path(audio_file_path).suffix == output_file_type): return

    # convert all audio file types to the output format
    # the top guard clause should skip over any file format in the list below
    # if they input file is already this format
# audio file typ conversions
    if (pathlib.Path(audio_file_path).suffix == ".wav"): convert_file(audio_file_path); # print("wav")
    if (pathlib.Path(audio_file_path).suffix == ".mp3"): convert_file(audio_file_path); # print("mp3")
    if (pathlib.Path(audio_file_path).suffix == ".flac"): convert_file(audio_file_path); # print("flac")

# video file type conversions:
    # if (pathlib.Path(audio_file_paths).suffix == ".m4a"): convert_file(audio_file_paths[i])
    # if (pathlib.Path(audio_file_paths).suffix == ".mp4"): convert_file(audio_file_paths[i])

    os.remove(audio_file_path)




    return

In [ ]:

# call all functions in here

for i in range(len(sub_folder_paths)):

    audio_files = pathlib.Path(sub_folder_paths[i]).glob("**/*") #generator object
    audio_files

    audio_file_paths =  [x for x in audio_files if x.is_file()] #PosixPath list
    print(len(audio_file_paths))
    for x in range(len(audio_file_paths)):
        check_file_type(audio_file_paths[x])
        print(x + i)  
    
    # print(audio_file_paths[0])

    


# working above on new version

---


In [ ]:
# input the bucket directory here
# must be a parent directory with folders of audio files
bucket ="/Volumes/UGREEN Samsung SSD/project echo audio/test data/test_1"
species_bucket = os.listdir(bucket)
species_bucket

In [ ]:
# writes creates a dataframe of species (folder names)




species_bucket = pd.DataFrame(species_bucket, columns = ["animal species"])
species_bucket.index = np.arange(1, len(species_bucket) + 1)
species_bucket = species_bucket.sort_values("animal species")
species_bucket = species_bucket.iloc[1:]
species_bucket.index = np.arange(1, len(species_bucket) + 1)
print("files: {}".format(len(species_bucket)))
species_bucket.head()

In [ ]:
## extract species as dataframe
# function to crearte a dataframe of all species (folders) in the bucket, listed alphabetically
def species_as_dataframe(folder_filepath):
    species_list = os.listdir(folder_filepath)
    species_dataframe = pd.DataFrame(species_list, columns = ["animal species"])
    species_dataframe.index = np.arange(1, len(species_dataframe) + 1)
    species_dataframe = species_dataframe.sort_values("animal species")
    species_dataframe = species_dataframe.iloc[1:]
    species_dataframe.index = np.arange(1, len(species_dataframe) + 1)
    return species_dataframe 

species_as_dataframe(bucket)


In [ ]:
# function to crearte a dataframe of full directory names of species in the bucket, listed alphabetically

def file_paths_as_df(bucket):  # maybe change to receive the column value too
    bucket_folders = glob.glob(bucket+"/*")
    bucket_folders = pd.DataFrame(bucket_folders, columns = ["animal species folder paths"])
    bucket_folders = bucket_folders.sort_values("animal species folder paths")
    bucket_folders.index = np.arange(1, len(bucket_folders) + 1)
    bucket_folders.head()
    return bucket_folders
bucket_file_paths_as_dataframe = file_paths_as_df(bucket)
bucket_file_paths_as_dataframe

In [ ]:
# this takes a dataframe and changes it back to list
def file_paths_as_list(bucket_folders_dataframe):
    bucket_folders_list = bucket_folders_dataframe.values.tolist()
    bucket_folders_list = list(np.concatenate(bucket_folders_list).flat)
    bucket_folders_list[0:1]
    return bucket_folders_list

In [ ]:
bucket_file_paths_as_list = file_paths_as_list(file_paths_as_df(bucket))
bucket_file_paths_as_list

In [ ]:
# this creates a .wav version of a file past to it
# it will save with the same name
# it will overwrite if an existing .wav file exists with the same name

def convert_individual_file(file_path):
    parent_path = pathlib.PurePath(file_path).parent    
    file_name = pathlib.Path(file_path).stem   
    output_file_path = str(parent_path) + "/" + file_name + ".mp3"
    # write the new wav file forcing overwrite 
    subprocess.call(['ffmpeg', '-y', '-i', file_path,
                 output_file_path])
    return

In [ ]:
# this function checks the filepath passed to it and checks the filetype
# if it is one of the listed filetypes ot be converted, it will make a new version 
# as a .wav and delete the original

def check_for_conversion(i, species_filepath):  
    audio_file_paths = glob.glob(species_filepath+"/*")

    for i in range(len(audio_file_paths)):
        if pathlib.Path(audio_file_paths[i]).suffix == ".wav":
            convert_individual_file(audio_file_paths[i]) # save a new copy as wav in this directory
            os.remove(audio_file_paths[i])

        elif pathlib.Path(audio_file_paths[i]).suffix == ".flac":
            convert_individual_file(audio_file_paths[i]) # save a new copy as wav in this directory
            os.remove(audio_file_paths[i])

        elif pathlib.Path(audio_file_paths[i]).suffix == ".m4a":
            convert_individual_file(audio_file_paths[i]) # save a new copy as wav in this directory
            os.remove(audio_file_paths[i])

        elif pathlib.Path(audio_file_paths[i]).suffix == ".mp4":
            convert_individual_file(audio_file_paths[i]) # save a new copy as wav in this directory
            os.remove(audio_file_paths[i])
    return

In [ ]:
### working here ###

'''
pathlib.PurePath would work much better here I'm sure ...
'''

def convert_files(bucket_file_path):
    print("Folders scanned")
    print(".........")
    for i in range(len(species_bucket)):
        bucket_species_dataframe = species_as_dataframe(bucket_file_path)
        bucket_file_paths_as_list = file_paths_as_list(file_paths_as_df(bucket_file_path))
        current_species_file_pathame = bucket_file_paths_as_list[i]
        check_for_conversion(i, current_species_file_pathame)
    print(bucket_species_dataframe)
    print(".........")
    print("Conversion Complete")




In [ ]:
convert_files(bucket)

In [ ]:
# run a second time to generate a clean output
convert_files(bucket)